In [6]:
#Importing the Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

#### Part 1: Data Preprocessing

In [8]:
#Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [9]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [10]:
print(y)

[1 0 1 ... 1 1 0]


In [13]:
## Encoding Categorical Data
#Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [14]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [15]:
#One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [16]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [17]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [18]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#### Part 2: Building The ANN

In [19]:
#Initializing the ANN
ann = tf.keras.models.Sequential()

In [20]:
#Adding the Input Layer and the first Hidden Layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [21]:
#Adding the second Hidden Layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [22]:
#Adding the Output Layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#### Part 3: Training The ANN

In [23]:
#Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [24]:
#Training the ANN on the Training Set
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5690 - accuracy: 0.7594
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4724 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4421 - accuracy: 0.7964
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4295 - accuracy: 0.7997
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4238 - accuracy: 0.8023
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4200 - accuracy: 0.8058
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4161 - accuracy: 0.8098
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4118 - accuracy: 0.8169
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4070 - accuracy: 0.8249
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4014 - accura

#### Part 4: Making the predictions and evaluating the model

In [25]:
#Predicting the Test Set results
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [26]:
#Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1523   72]
 [ 209  196]]


0.8595

#### Predicting the Result of a Single Observation

I will use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

In [27]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 33ms/step
[[False]]


My ANN model predicts that this customer stays in the bank!